## Script que gera os arquivos base para o aprendizado do Corruptometro.

In [2]:
import pandas as pd
import numpy as np
import unicodedata
path = '../../data/'
def padronizaTexto(texto):
    """
    Remove acentos e coloca em caixa alta todo o texto que necessita ser padronizado.
    
    """
    return unicodedata.normalize('NFKD', texto.decode('utf-8')).encode('ASCII', 'ignore').upper()

### Manipulação das informações da UNIDADE GESTORA

In [4]:
ANO_GESTAO_2009, ANO_GESTAO_2012, ANO_GESTAO_2013, ANO_GESTAO_2016 = 2009, 2012, 2013, 2016

def mudaAnoUgestora(ano):
    """
    Troca o valor do dt_Ano pelo ano do inicio da gestão.
    Isso será necessário para unir com a tabela de eletos e a de inidoneidade no futuro.
    """
    inicio_gestao_2009, inicio_gestao_2013 = 2009, 2013
    if ano >= ANO_GESTAO_2009 and ano <= ANO_GESTAO_2012:
        return str(inicio_gestao_2009)
    elif ano >= ANO_GESTAO_2013 and ano <= ANO_GESTAO_2016:
        return str(inicio_gestao_2013)

def nomeUgestora(gestora):
    """
    Remove a frase 'Prefeitura Municipal de'todos os nomes de unidade gestora.

    """
    QUANT_CARACTER_REMOVE = 24
    return padronizaTexto(gestora[QUANT_CARACTER_REMOVE:])

def manipulaUnidadeGestora():
    """
    Retorna o código da unidade gestora, o ano do inicio da gestão e a unidade gestora para todas as prefeituras
    a partir de 2009.

    """    
    path_ges = path+'codigo_ugestora.csv'
    ud_gestoras = pd.read_csv(path_ges) 
    ud_gestoras_pref = ud_gestoras[(ud_gestoras['dt_Ano'] > 2008) & 
                                   (ud_gestoras['de_Ugestora'].str.contains("Prefeitura"))]
    func_anos, func_nome = np.vectorize(mudaAnoUgestora), np.vectorize(nomeUgestora)
    anos, u_gestoras = ud_gestoras_pref['dt_Ano'].values, ud_gestoras_pref['de_Ugestora'].values
    nov_anos, nov_u_gestoras = func_anos(anos), func_nome(u_gestoras)
    ud_gestoras_pref.loc[:,('dt_Ano')], ud_gestoras_pref.loc[:,('de_Ugestora')] = nov_anos, nov_u_gestoras
    
    condition = ud_gestoras_pref['de_Ugestora'] == 'TACIMA'
    ud_gestoras_pref.loc[condition,'de_Ugestora'] = 'CAMPO DE SANTANA'
    
    return ud_gestoras_pref[['cd_Ugestora','dt_Ano','de_Ugestora']].drop_duplicates()

ud_gestoras_pref_pos_2008 = manipulaUnidadeGestora()
print 'Numero de Gestoes:', len(ud_gestoras_pref_pos_2008)
print 'Parte do Arquivo:'
ud_gestoras_pref_pos_2008.head()

Numero de Gestoes: 446
Parte do Arquivo:


,cd_Ugestora,dt_Ano,de_Ugestora
1666,201001,2009,AGUA BRANCA
1670,201001,2013,AGUA BRANCA
1680,201002,2009,AGUIAR
1684,201002,2013,AGUIAR
1694,201003,2009,ALAGOA GRANDE


## Manipulação da base de dados de candidatos eleitos 2008-2012

In [7]:
def anoAposEleicao(ano):
    """
    Para termos como referencia o ano em que o prefeito começou sua gestão. Não o ano em que ele foi eleito.

    """
    ano = str(ano)
    ELEICAO_2008, GESTAO_2009, ELEICAO_2012, GESTAO_2013 = '2008', '2009', '2012', '2013'
    if ano == ELEICAO_2008:
        return GESTAO_2009
    elif ano == ELEICAO_2012:
        return GESTAO_2013

def adicionarLinhasComplementares():
    """
    Muitos dos municipios paraibanos não constam na lista original dos arquivos de candidatos eleitos. Outros 
    """
    complemento = pd.DataFrame([[2012,"NATUBA" ,"JOSÉ LINS DA SILVA FILHO"],
                                [2008,"NATUBA","Josevaldo Alves da Silva"],
                                [2012,"MULUNGU","JOANA DARC RODRIGUES BANDEIRA FERRAZ"],
                                [2008,"MULUNGU","José Leonel de Moura"],
                                [2012,"MONTEIRO","EDNACÉ ALVES SILVESTRE HENRIQUE"],
                                [2008,"MONTEIRO","Ednacé Alves Silvestre Henrique"],
                                [2012,"MONTE HOREBE","CLAUDIA APARECIDA DIAS"],
                                [2008,"MONTE HOREBE" ,"Erivan Dias Guarita"],
                                [2012,"MONTADAS","JAIRO HERCULANO DE MELO"],
                                [2008,"MONTADAS" ,"Lindembergue Souza Silva"],
                                [2012,"MOGEIRO","ANTONIO JOSÉ FERREIRA"],
                                [2008,"MOGEIRO" ,"Antonio José Ferreira"],
                                [2012,"MATURÉIA","DANIEL DANTAS WANDERLEY"],
                                [2008,"MATURÉIA" ,"Daniel Dantas Wanderley"],
                                [2012,"MATO GROSSO","RAELLYSON RODRIGO OLIVEIRA MONTEIRO"],
                                [2008,"MATO GROSSO","Katsonara Soares de Andrade Monteiro"],
                                [2012,"MATINHAS","MARIA DE FÁTIMA SILVA"],
                                [2008,"MATINHAS" ,"Jose Costa Aragao Junior"],
                                [2012,"MATARACA","OLIMPIO DE ALENCAR ARAUJO BEZERRA"],
                                [2008,"MATARACA","Joao Madruga da Silva"],
                                [2012,"MASSARANDUBA","JOANA DARC QUEIROGA MENDONCA COUTINHO"],
                                [2008,"MASSARANDUBA" ,"Paulo Fracinette de Oliveira"],
                                [2012,"MARIZÓPOLIS","JOSÉ VIEIRA DA SILVA"],
                                [2008,"MARIZÓPOLIS" ,"Jose Vieira da Silva"],
                                [2012,"MARI","MARCOS AURELIO MARTINS DE PAIVA"],
                                [2008,"MARI" ,"Antonio Gomes da Silva"],
                                [2008,"MALTA" ,"Ajácio Gomes Wanderley"],
                                [2012,"MALTA" ,"Manoel Benedito de Lucena Filho"],
                                [2008,"MAMANGUAPE" ,"Eduardo Carneiro de Brito"],
                                [2012,"MAMANGUAPE" ,"Eduardo Carneiro de Brito"],
                                [2008,"MANAIRA" ,"José Simão de Sousa"],
                                [2012,"MANAIRA" ,"Jose Simao de Sousa"],
                                [2012,"LUCENA","MARCELO SALES DE MENDONÇA"],
                                [2008,"LUCENA" ,"Antonio Mendonça Monteiro Junior"],
                                [2012,"JOCA CLAUDINO","LUCRECIA ADRIANA DE ANDRADE BARBOSA DANTAS"],
                                [2008,"JOCA CLAUDINO" ,"Lucrecia Adriana de Andrade Barbosa Dantas"],
                                [2008,"CAMPO DE SANTANA","TARGINO PEREIRA DA COSTA NETO"],
                                [2012,"CAMPO DE SANTANA","Erivan Bezerra Daniel"]],
                               columns=list(['dt_Ano','de_Ugestora','NOME_CANDIDATO']))
    func_nome, func_ano_pos_eleicao =  np.vectorize(padronizaTexto), np.vectorize(anoAposEleicao)
    gestoras, nomes_cand = complemento['de_Ugestora'].values,complemento['NOME_CANDIDATO'].values
    anos = complemento['dt_Ano'].values
    n_gestoras, n_nomes_cand, n_anos = func_nome(gestoras), func_nome(nomes_cand), func_ano_pos_eleicao(anos)
    complemento.loc[:,('de_Ugestora')] = n_gestoras
    complemento.loc[:,('NOME_CANDIDATO')] = n_nomes_cand
    complemento.loc[:,('dt_Ano')] = n_anos

    return complemento

def manipulaCandidatosEleitos():
    """
    Manipula os dados das eleicoes para selecionar apenas os prefeitos eleitos entre 2008 e 2012.
    Ajustando o ano em que começaram seu mandato.
    
    Os arquivos tiveram de ser compactados para poderem estar em um tamanho aceitável para o github.

    """
    COD_PREFEITO = '11' 
    path_cand_2008, path_cand_2012 = path+'candidate_local_2008.tar.bz2', path+'candidate_local_2012.tar.bz2'
    cand_2008, cand_2012 = pd.read_csv(path_cand_2008, dtype='unicode', compression='bz2'), pd.read_csv(path_cand_2012, dtype='unicode', compression='bz2')
    eleicoes2008_2012 = [cand_2008,cand_2012]
    cand_2008_2012 = pd.concat(eleicoes2008_2012)
    
    cand_2008_2012_PB_ELEITOS = cand_2008_2012[(cand_2008_2012['SIGLA_UF'] == 'PB') &
                                               (cand_2008_2012['DESC_SIT_TOT_TURNO'] == 'ELEITO') &
                                              (cand_2008_2012['CODIGO_CARGO'] == COD_PREFEITO)]
    
    func_ano_pos_eleicao = np.vectorize(anoAposEleicao)
    func_nome_prefeit = np.vectorize(padronizaTexto)
    func_nome_cidade = np.vectorize(padronizaTexto)
    
    anos_eleic = cand_2008_2012_PB_ELEITOS['ANO_ELEICAO'].values
    nomes_pref_eleit = cand_2008_2012_PB_ELEITOS['NOME_CANDIDATO'].values
    nomes_cid = cand_2008_2012_PB_ELEITOS['DESCRICAO_UE'].values
    
    anos_pos_eleicao = func_ano_pos_eleicao(anos_eleic) 
    nomes_pref_eleitos =  func_nome_prefeit(nomes_pref_eleit) 
    n_nomes_cid = func_nome_cidade(nomes_cid)
    
    cand_2008_2012_PB_ELEITOS.loc[:,('ANO_ELEICAO')] = anos_pos_eleicao
    cand_2008_2012_PB_ELEITOS.loc[:,('NOME_CANDIDATO')] = nomes_pref_eleitos
    cand_2008_2012_PB_ELEITOS.loc[:,('DESCRICAO_UE')] = n_nomes_cid
    
    cand_2008_2012_PB_ELEITOS.rename(columns={'DESCRICAO_UE': 'de_Ugestora', 'ANO_ELEICAO':'dt_Ano'}, inplace=True)
    complemento = adicionarLinhasComplementares()
    cand_2008_2012_PB_ELEITOS_INIC = cand_2008_2012_PB_ELEITOS[['dt_Ano','de_Ugestora', 'NOME_CANDIDATO']]
    cand_2008_2012_completo = [cand_2008_2012_PB_ELEITOS_INIC,complemento]
    
    cand_2008_2012_completo = pd.concat(cand_2008_2012_completo).drop_duplicates()
    
    condition = cand_2008_2012_completo['de_Ugestora'] == 'SAO DOMINGOS DE POMBAL'
    cand_2008_2012_completo.loc[condition,'de_Ugestora'] = 'SAO DOMINGOS'
    cand_2008_2012_completo = cand_2008_2012_completo[~cand_2008_2012_completo['de_Ugestora'].isin(['SANTAREM','TACIMA'])]
    
    return cand_2008_2012_completo
       
"""
Os municipios: NOVA OLINDA, SAO JOSE DO SABUGI, SOLEDADE aparecem repetidamente para uma mesma eleicao. 
Provavelmente pela posse de outros prefeitos diferente os eleitos. Sendo assim, há 223 municipios que 
para as eleicoes de 2008 e 2012 somam-se 446. Com a repetição dos anos para esses municipios.
Além disso: SAO DOMINGOS DE POMBAL virou SAO DOMINGOS, Santarém virou Joca Claudino, Tacima virou Campo de Santana
"""
cand_2008_2012_PB_ELEITOS = manipulaCandidatosEleitos()
print 'Numero de Gestoes:', len(cand_2008_2012_PB_ELEITOS)
print 'Parte do Arquivo:'
cand_2008_2012_PB_ELEITOS.head()

Numero de Gestoes: 449
Parte do Arquivo:


,dt_Ano,de_Ugestora,NOME_CANDIDATO
184412,2009,AGUIAR,MANOEL BATISTA GUEDES FILHO
184438,2009,ALAGOA GRANDE,JOAO BOSCO CARNEIRO JUNIOR
184530,2009,ALAGOA NOVA,KLEBER HERCULANO DE MORAES
184581,2009,ALAGOINHA,ALCIONE MARACAJA DE MORAIS BELTRAO
184633,2009,ALCANTIL,JOSE MILTON RODRIGUES


In [6]:
"""
Mostra a padronização de ambos os arquivos quanto a unidade gestora.
"""
a =  set(cand_2008_2012_PB_ELEITOS['de_Ugestora'].unique())
b = set(ud_gestoras_pref_pos_2008['de_Ugestora'].unique())
print(a-b)
print(b-a)

set([])
set([])


## Amplia a tabela Sagres com as informacoes Eleitos por Ano.

In [7]:
sagres_pref_eleit_por_ano = pd.merge(ud_gestoras_pref_pos_2008,cand_2008_2012_PB_ELEITOS,on=['de_Ugestora', 'dt_Ano'])
sagres_pref_eleit_por_ano = sagres_pref_eleit_por_ano.drop_duplicates()
print len(sagres_pref_eleit_por_ano)
sagres_pref_eleit_por_ano.head()

449


,cd_Ugestora,dt_Ano,de_Ugestora,NOME_CANDIDATO
0,201001,2009,AGUA BRANCA,AROUDO FIRMINO BATISTA
1,201001,2013,AGUA BRANCA,TARCISIO ALVES FIRMINO
2,201002,2009,AGUIAR,MANOEL BATISTA GUEDES FILHO
3,201002,2013,AGUIAR,MANOEL BATISTA GUEDES FILHO
4,201003,2009,ALAGOA GRANDE,JOAO BOSCO CARNEIRO JUNIOR


## Manipula a base de gestores condenados pelo TRE.

In [14]:
def anoPublPGestao(ano):
    """
    Transforma o ano da data de publicação em ano do início de mandato.
    
    """
    #Acrescenta 20 ao final da data 12/03/09 - 20+09 = 2009
    ano = int('20'+ano[6:])
    return mudaAnoUgestora(ano)
    
    
def manipulaCandidatosCondenadosTRE():
    """
    
    """
    path_jul = path+'contas_julgadas_irregulares_TRE_2016.csv'
    contas_jul_irr = pd.read_csv(path_jul)
    contas_jul_irr = contas_jul_irr[contas_jul_irr['JURISDICIONADO'].str.contains("Prefeitura")]

    func_padr_nome_resp = np.vectorize(padronizaTexto)
    func_padr_nome_ugest = np.vectorize(nomeUgestora) 
    func_padr_ano = np.vectorize(anoPublPGestao)  
    
    nome_resp = func_padr_nome_resp(contas_jul_irr['RESPONSÁVEL'].values)
    nome_ugest = func_padr_nome_ugest(contas_jul_irr['JURISDICIONADO'].values)
    data_public_gest = func_padr_ano(contas_jul_irr['DATA PUBLICAÇÃO'].values) 
    
    contas_jul_irr.loc[:,('RESPONSÁVEL')] = nome_resp
    contas_jul_irr.loc[:,('JURISDICIONADO')] = nome_ugest
    contas_jul_irr.loc[:,('DATA PUBLICAÇÃO')] = data_public_gest
    
    contas_jul_irr.rename(columns={'JURISDICIONADO':'de_Ugestora', 'DATA PUBLICAÇÃO':'dt_Ano'}, inplace=True)
    return contas_jul_irr

candidatosCondenadosTRE = manipulaCandidatosCondenadosTRE()
candidatosCondenadosTRE = candidatosCondenadosTRE[['dt_Ano','de_Ugestora','ITEM','RESPONSÁVEL']]
print len(candidatosCondenadosTRE['de_Ugestora'].unique())
candidatosCondenadosTRE.head()

138


,dt_Ano,de_Ugestora,ITEM,RESPONSÁVEL
0,2009,PUXINANA,1,ABELARDO ANTONIO COUTINHO
1,2009,PUXINANA,2,ABELARDO ANTONIO COUTINHO
2,2009,PUXINANA,3,ABELARDO ANTONIO COUTINHO
3,2013,PUXINANA,4,ABELARDO ANTONIO COUTINHO
4,2013,PUXINANA,5,ABELARDO ANTONIO COUTINHO


## Junção entre as tabelas do TRE e Sagres para gerar as gestões condenáveis.

In [15]:
def classificaEleito(processo):
    """
    Se a coluna eh vazia entao nao ha processos. Em caso contrario ha processo e eh classificado como irregular.
    """
    if pd.isnull(processo):
        return 'NAO IRREGULAR'
    return 'IRREGULAR'

cand_cond_TREXSagres_Gest = pd.merge(sagres_pref_eleit_por_ano, candidatosCondenadosTRE, how='left', on=['de_Ugestora','dt_Ano'])
cand_cond_TREXSagres_Gest.rename(columns={'NOME_CANDIDATO':'Eleito'}, inplace=True)
func_class_gest = np.vectorize(classificaEleito)
classes = func_class_gest(cand_cond_TREXSagres_Gest['ITEM'].values)
cand_cond_TREXSagres_Gest['Classe'] = classes
cand_cond_TREXSagres_Gest = cand_cond_TREXSagres_Gest[['cd_Ugestora','dt_Ano','de_Ugestora','Eleito','Classe']].drop_duplicates()
print len(cand_cond_TREXSagres_Gest)
cand_cond_TREXSagres_Gest.to_csv(path+'TRE_Sagres_Resp_Eleito_Final.csv',sep=',',encoding='utf-8', index=False)

449


In [16]:
cand_cond_TREXSagres_Gest.head()

,cd_Ugestora,dt_Ano,de_Ugestora,Eleito,Classe
0,201001,2009,AGUA BRANCA,AROUDO FIRMINO BATISTA,IRREGULAR
1,201001,2013,AGUA BRANCA,TARCISIO ALVES FIRMINO,IRREGULAR
3,201002,2009,AGUIAR,MANOEL BATISTA GUEDES FILHO,NAO IRREGULAR
4,201002,2013,AGUIAR,MANOEL BATISTA GUEDES FILHO,NAO IRREGULAR
5,201003,2009,ALAGOA GRANDE,JOAO BOSCO CARNEIRO JUNIOR,NAO IRREGULAR
